# Training an intents classification model

Define constants

In [1]:
from os import getcwd, path
import sys
import numpy as np

BASE_PATH = path.dirname(getcwd())
sys.path.append(BASE_PATH)

DATA_UTILS = path.join(BASE_PATH, 'common/data_utils.py')
TRAIN_PATH = path.join(BASE_PATH, 'kc_data.json')
CLASSES_FILE = path.join(BASE_PATH, 'classes.json')

In [2]:
exec(open(DATA_UTILS).read())

Use functions from the utils to extract and preprocess the training data
Refer to `kc_data.json` for the sample data format
`get_data_pairs` is then used to parse data into a tuple of `([list_of_sentences], [list_of_labels])`

In [3]:
X_train, y_train = get_data_pairs(data_from_json(TRAIN_PATH))

Save the classes descriptions as json

In [4]:
import torch
torch.__version__

'0.4.1'

Start training the classification model and save

In [5]:
from text_classification.ensemble.model import EnsembleWrapper
from text_classification.ensemble.train import EnsembleLearner
from common.callbacks import PrintLoggerCallback

model = EnsembleWrapper()
learner = EnsembleLearner(model)

In [6]:
learner.fit(
    training_data=(X_train, y_train),
    batch_size=64
)

Iteration 1, loss = 7.15302609
Iteration 2, loss = 6.75358355
Iteration 3, loss = 6.29351703
Iteration 4, loss = 5.69058522
Iteration 5, loss = 5.03504465
Iteration 6, loss = 4.38733521
Iteration 7, loss = 3.78765651
Iteration 8, loss = 3.25905688
Iteration 9, loss = 2.81315959
Iteration 10, loss = 2.43811695
Iteration 11, loss = 2.12791411
Iteration 12, loss = 1.87443303
Iteration 13, loss = 1.66635126
Iteration 14, loss = 1.49074848
Iteration 15, loss = 1.34690295
Iteration 16, loss = 1.22706702
Iteration 17, loss = 1.12538576
Iteration 18, loss = 1.03663595
Iteration 19, loss = 0.96185606
Iteration 20, loss = 0.89732125
Iteration 21, loss = 0.84072673
Iteration 22, loss = 0.78694415
Iteration 23, loss = 0.74238454
Iteration 24, loss = 0.70268385
Iteration 25, loss = 0.66824649
Iteration 26, loss = 0.63265816
Iteration 27, loss = 0.60534400
Iteration 28, loss = 0.57856664
Iteration 29, loss = 0.55380625
Iteration 30, loss = 0.53051573
Iteration 31, loss = 0.51216545
Iteration 32, los

In [7]:
model(['good morning!'])

/Users/2359media/anaconda/envs/botbot-nlp/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


[[{'confidence': 0.9557663202285767, 'intent': 'Greeting - Morning'},
  {'confidence': 0.035246167331933975, 'intent': 'SmallTalk - Goodbye'},
  {'confidence': 0.006699211895465851, 'intent': 'General - IAmTired'},
  {'confidence': 0.0009792317869141698,
   'intent': 'Diet - What Can Eat - Snack'},
  {'confidence': 0.00041592770139686763,
   'intent': 'Postnatal - Confinement - Practices'}]]

In [8]:
model(['my baby\'s head is too big'])

/Users/2359media/anaconda/envs/botbot-nlp/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


[[{'confidence': 0.8279853463172913,
   'intent': 'Trimester - Third - How big IT'},
  {'confidence': 0.029442988336086273,
   'intent': 'Trimester - Third - How big'},
  {'confidence': 0.027228983119130135,
   'intent': 'Trimester - First - How big'},
  {'confidence': 0.023028625175356865,
   'intent': 'Trimester - Second - how big'},
  {'confidence': 0.011571577750146389, 'intent': 'Medical - Belly Is Big'}]]